Vectorstore-backed Memory

This means storing the personality preferences in a data structure that's easy to look up. In the case of LLMs, since Q&A is unstructured data and queries, a vector database might come to mind. This is not the only option (can always convert unstructured queries to structured, and query data in SQL or other ways).

Pinecone setup to store user's personality bank , and only query the relevant parts when the assistant looks at the user's query.

First, we set up Pinecone as our vector database.

Step one is to install Pinecone.

In [ ]:
pip install pinecone-client --quiet

In [ ]:
PINECONE_API_KEY = getpass.getpass('Enter your Pinecone API key: ')

setup Pinecone SDK client. using Pinecone's new serverless architecture (free lol)

In [ ]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

create an index. An index for a vector database is a way to organize embeddings in an efficient manner for search. Popular indexing algorithms are: HNSW, IVF, etc.. I use HNSW with Supabase (which is just a Postgres table and pg-vector extension underneath)

In [ ]:
index_name = "user-preferences"

EMBEDDINGS_DIMENSIONS = 512

# Just checking to see if this index already exist
existing_indexes = pc.list_indexes().names()

# If index doesn't exist yet, then delete it and create one (we're starting from scratch)
if index_name in existing_indexes:
  pc.delete_index(index_name)
  print("Deleted index.")

pc.create_index(
    name=index_name,
    dimension=EMBEDDINGS_DIMENSIONS,
    metric="cosine",
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)
print("Created index.")

index = pc.Index(index_name)